# Malware Analysis & Triage Kit
This notebook performs the initial stages of immediate malware triage.

## How To
Take your malware specimen and drop it into the `dropbox` directory. The notebook will walk you through the stages of initial analysis.

At the end of this process, you will have a triage report in the `saved-specimens` diretory. This report includes findings from initial triage, including the defanged specimen in a password-proteced Zip file and static analysis artifacts.

### Imports and Setup

In [1]:
# Import main.py for malware analysis scirpt
from main import *
from hashlib import *
import sys
import os
from getpass import getpass
from virus_total_apis import PublicApi as VirusTotalPublicApi
import json

In [2]:
samples = enum_malware_samples()

SamplePositive.txt
SampleNegative.txt


In [3]:
specimen_dirs = create_specimen_dirs(samples)

In [4]:
saved_samples = move_samples(samples, specimen_dirs)

## File Hashes

In [5]:
hashes = get_hashes(saved_samples)

[+] Sample: saved-specimens/09-25-2021-112446_SamplePositive.txt/SamplePositive.txt SHA256 SUM: 275a021bbfb6489e54d471899f7db9d1663fc695ec2fe2a2c4538aabf651fd0f
[+] Sample: saved-specimens/09-25-2021-112446_SampleNegative.txt/SampleNegative.txt SHA256 SUM: 5814318064326228845b4b2e8f4ae11d0d249475e2845e5eda05fb84bbb19bce


## Defang Sample

In [6]:
#TODO
defang_samples(saved_samples)

Defang!


## Strings, FLOSS, and StringSifter
TODO: Currently working on solving poetry add/install issue with remote repositories (`poetry add https://github.com/fireeye/flare-floss/zipball/master` doesn't work

### StringSifter
StringSifter is a FLARE developed tool that uses an ML model to rank a binary's strings by relevance to malware analysis.

In [7]:
pull_strings(saved_samples, specimen_dirs)

['saved-specimens/09-25-2021-112446_SamplePositive.txt/SamplePositive.txt', 'saved-specimens/09-25-2021-112446_SampleNegative.txt/SampleNegative.txt']


[*] Input your desired minimum string length [default is 4, 6-8 is recommended] >  8


[*] Written to outfile: StringSifter-Out.log
[*] Written to outfile: StringSifter-Out.log


## VT Analysis
Submit samples to Virus Total and generate a malicious confidence level.

In [5]:
VT_API_KEY = getpass("Enter VirusTotal API Key (blank if none): ")

Enter VirusTotal API Key (blank if none):  


In [5]:
if VT_API_KEY:
    vt = VirusTotalPublicApi(VT_API_KEY)
else:
    print(info + "No VT API Key. Skipping...")

NameError: name 'VT_API_KEY' is not defined

In [ ]:
if vt:
    for hash in hashes:
        try:
            print(info + "SHA25sum: " + hash)
            #res = vt.get_file_report(hash)
            # Test case with known bad hash:
            res = vt.get_file_report(hash)

            #print([r["result"] for r in res["results"]["scans"].values()])

            # TODO: do something here to parse response code dict object. if > 0, success. Else, respond with "nothing returned from VT"
            # Right now, a negative result returns Error: 'scans' which could be handled more gracefully
            conf = malicious_confidence(res)
            print(info + "Confidence level: " + str(conf))
            criticality =  determine_criticality(conf)
            
            
        except Exception as e:
            print(printError + "Error: " + str(e))
            pass
else:
    print(info + "No VT Key. Skipping...")